# API Face Samples

## Objective

To develop a Python-based application that detects and verifies faces within a face collection (large face list or large person group) using Azure Face SDK. The application will cover three scenarios:

- **Image + Large Face List**: Detect faces in the image and check if the largest detected face matches any face in the given large face list.
- **Face ID + Large Face List**: Verify if the provided face ID matches any face in the given large face list.
- **Image + Large Person Group**: Detect faces in the image and identify if the largest detected face belongs to any person in the given large person group.
- **Face ID + Large Person Group**: Identify if the provided face ID belongs to any person in the given large person group.

The function returns relevant identifiers and confidence scores, allowing users to determine the match accuracy and identity of faces in different contexts.


In [ ]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.vision.face import FaceClient
from azure.ai.vision.face.models import FaceDetectionModel, FaceRecognitionModel

def find_faces_in_collection(subscription_key, endpoint, image_path=None, face_id=None, large_face_list_id=None, large_person_group_id=None):   
    def detect_face(image_path):
        with FaceClient(endpoint, AzureKeyCredential(subscription_key), headers = {"X-MS-AZSDK-Telemetry": "sample=find-faces-in-collection"} ) as face_client:
            with open(image_path, 'rb') as image_data:
                detected_faces = face_client.detect(
                        image_content=image_data.read(),
                        detection_model=FaceDetectionModel.DETECTION03,
                        recognition_model=FaceRecognitionModel.RECOGNITION04,
                        return_face_id=True,
                    )
            return [face.face_id for face in detected_faces]

    def check_face_id_in_face_list(face_id, large_face_list_id):
        with FaceClient(endpoint, AzureKeyCredential(subscription_key), headers = {"X-MS-AZSDK-Telemetry": "sample=find-faces-in-collection"} ) as face_client:
            similar_faces = face_client.find_similar_from_large_face_list(
                face_id=face_id,
                large_face_list_id=large_face_list_id,
                max_num_of_candidates_returned=1
            )  
            if similar_faces and similar_faces[0].confidence:
                persisted_face_id = similar_faces[0].persisted_face_id
                confidence = similar_faces[0].confidence
                return {"persistedFaceId": persisted_face_id, "confidence": confidence}
            else:
                return {}

    def check_face_id_in_person_group(face_id, large_person_group_id):
        with FaceClient(endpoint, AzureKeyCredential(subscription_key), headers = {"X-MS-AZSDK-Telemetry": "sample=find-faces-in-collection"} ) as face_client:
            identify_results = face_client.identify_from_large_person_group(
                face_ids=[face_id],
                large_person_group_id=large_person_group_id
            )
            if identify_results and identify_results[0].candidates:
                person_id = identify_results[0].candidates[0].person_id
                confidence = identify_results[0].candidates[0].confidence
                return {"personId": person_id, "confidence": confidence}
            else:
                return {}
    
    if image_path:
        face_ids = detect_face(image_path)
        if not face_ids:
            raise ValueError("No face detected in the image.")
        else:
            face_id = face_ids[0]
    
    if large_face_list_id:
        return check_face_id_in_face_list(face_id, large_face_list_id)
    elif large_person_group_id:
        return check_face_id_in_person_group(face_id, large_person_group_id)
    else:
        raise ValueError("Either large_face_list_id or large_person_group_id must be provided.")

# Example usage:
FACE_KEY = os.environ["FACE_API_KEY"]
FACE_ENDPOINT = os.environ["FACE_ENDPOINT_URL"]
image_path = 'path_to_image.jpg'
face_id = 'your_face_id'
large_face_list_id = 'your_large_face_list_id'
large_person_group_id = 'your_large_person_group_id'

# Image + Large Face List
print(find_faces_in_collection(FACE_KEY, FACE_ENDPOINT, image_path=image_path, large_face_list_id=large_face_list_id))

# Face ID + Large Face List
print(find_faces_in_collection(FACE_KEY, FACE_ENDPOINT, face_id=face_id, large_face_list_id=large_face_list_id))

# Image + Large Person Group
print(find_faces_in_collection(FACE_KEY, FACE_ENDPOINT, image_path=image_path, large_person_group_id=large_person_group_id))

# Face ID + Large Person Group
print(find_faces_in_collection(FACE_KEY, FACE_ENDPOINT, face_id=face_id, large_person_group_id=large_person_group_id))